# Compressing sparse matrix using auto encoder

* problem statement : Showcase a solution for optimal representation of the m*n sparse matrix (10M x 30K) and near real-time retrieval from this data structure.

* solution propsed : using auto encoder trained on the data to compress it.

key highlights:

- Generalised solution that uses neural network to compress the data.

- Upto 50% decrease in memory required.

- Fast retrival than conventional methods

- Inspired by word embeddings in Natural language processing

# Training VAE on matrix data to compress it

**Author:** [Yoga Harshitha Duddukuri](https://www.linkedin.com/in/dyogaharshitha)<br>



**Algorithm:** Matrix (m x n) is tokenised and encoded with sinusoidal embedding. variational auto encoder is further trained on the embedding to compress it.

## Import modules

In [2]:
import math
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

## Data pipeline

Generate random matrix (10M x 30K). Pincode serviceability of suppliers over 30K pincodes.

In [3]:
dummy_data = np.random.randint(0,2,size=(1,30000) )
dummy_data[:,:15]

array([[1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0]])

Compress each row of size 30K into sinusoidal embeddings of 15K  ( 50% compression ratio achieved horizontally )

- 8 bits are grouped and result is tokenised to a embedding of length 4

In [4]:
dt = np.reshape(dummy_data, (8,-1))
bin_to_int = np.array([128,64,32,16,8,4,2,1] )
dt_tok = np.matmul(bin_to_int, dt)

print(dt_tok.shape)
# sinusoidal embedding

def positional_encoding(max_len, embedding_dim):
    position = np.arange(max_len, dtype=np.float32)
    angle_rates = 1 / np.power(10000, (2 * (np.arange(embedding_dim, dtype=np.float32) // 2)) / embedding_dim)
    angle_rads = np.expand_dims(position, -1) * angle_rates
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    pos_encoding = np.expand_dims(angle_rads, 0)
    return pos_encoding

def index_to_embedding(index, pos_encoding):

    return pos_encoding[:, index]

def embedding_to_index(embedding, pos_encoding):
    # Calculate the cosine similarity between the embedding and each positional encoding
    similarity = np.sum(embedding * pos_encoding, axis=-1)
    # Find the index with the highest similarity (nearest neighbor search)
    index = np.argmax(similarity, axis=-1)
    return index

max_len = 256
embedding_dim = 4
vocab_size = 256

# Create sinusoidal positional encoding
pos_encoding = positional_encoding(max_len, embedding_dim)

# Example index
index = np.array([[50],[51]])

# Convert index to sinusoidal embedding
embedding = index_to_embedding(index, pos_encoding)

# Convert embedding back to index
reconstructed_index = embedding_to_index(embedding, pos_encoding)

# Print results
print("Original Index:", index)
print("Sinusoidal Embedding:", embedding)
print("Reconstructed Index:", reconstructed_index)



(3750,)
Original Index: [[50]
 [51]]
Sinusoidal Embedding: [[[[-0.26237485  0.964966    0.47942555  0.87758255]]

  [[ 0.67022914  0.74215424  0.48817724  0.8727445 ]]]]
Reconstructed Index: [[50 51]]


data generation

In [5]:
# embedded data is (10M x 15K)  look up table of (256 x 4)

btch = 10
data = np.random.randint(0,256,size=(btch,3750))
dt_sin = index_to_embedding(data, pos_encoding)
dt_sin = np.reshape( dt_sin , (btch,-1) )
print(dt_sin.shape)

(10, 15000)


In [6]:
import csv
import numpy as np


n_chnk = 20 ; btch=1000
# Write data to CSV file in chunks
with open("/content/dummy_data.csv", "w") as csvfile:
    writer = csv.writer(csvfile)

    # Write data in chunks
    for i in range(n_chnk):
        data = np.random.randint(0,256,size=(btch,3750))
        chunk = data  # [i:i+chunk_size]
        writer.writerows(chunk)

print("CSV file created successfully.")
rows=0
for chunk in pd.read_csv("/content/dummy_data.csv", chunksize=1000):
  rows = rows + len(chunk)
print(rows)

CSV file created successfully.
19999


Till now data has been processed considering all the possible combinations that could occur and compressed making use of the memory allocation machanism, utilzing the resources to full extent.

- We perform agglomerative clustering to extract the pattern from the data. The data is further compressed, making use of clusters

In [12]:

# cluster suppliers or merchants by using Agglomerative clusering

import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.cluster import AgglomerativeClustering

# Example array
array = dt_sin #np.array([[1, 2], [5, 8], [1.5, 1.8], [8, 8], [1, 0.6], [9, 11]])

import pandas as pd
import numpy as np
from scipy.cluster.hierarchy import linkage, fcluster
from sklearn.preprocessing import StandardScaler

# Initialize an empty DataFrame to store the aggregated data
aggregated_data = pd.DataFrame()

# Define the chunk size
chunk_size = 1000 ;

threshold = 120.0 ; n_clusters= n_chnk  # Adjust as needed
clustering = AgglomerativeClustering(n_clusters=n_clusters)
# Iterate over each chunk in the dataset
for chunk in pd.read_csv("/content/dummy_data.csv", chunksize=chunk_size):

    # Perform hierarchical clustering on the current chunk

    Z = linkage(chunk, method='complete')

    # Apply flat clustering to assign each data point to a cluster
    # You can adjust the threshold to control the number of clusters

    labels = clustering.fit_predict(chunk)

    # Add the cluster labels to the DataFrame
    chunk['cluster_label'] = labels


    # Save the aggregated data to a CSV file or perform further analysis
    chunk.to_csv("/content/cluster_data.csv",mode='w' ,index=False)




In [14]:
import pandas as pd

# Parameters
chunk_size = 1000  # Chunk size
filter_column = 'cluster_label'  # Column to filter by
filter_value = 1  # Value to filter on

# Read the CSV file in chunks
chunks = pd.read_csv("/content/cluster_data.csv", chunksize=chunk_size)

# Initialize an empty list to store filtered chunks
filtered_chunks = []

# Filter each chunk by the desired column label and value
for chunk in chunks:
    filtered_chunk = chunk[chunk[filter_column] == filter_value]
    filtered_chunks.append(filtered_chunk)
    filtered_chunk.to_csv("/content/filtered_data.csv",mode='w', index=False)

# Concatenate the filtered chunks into a single DataFrame
filtered_data = pd.concat(filtered_chunks)


print("Filtered data saved successfully.")


Filtered data saved successfully.


# perform dimensionality reduction on each cluster

- we perform PCA on the array to reduce its dimensions, while retaining the data 99%.

- Since, the matrix is sparse and clustered to togther, PCA is able to further compress the data

- compression ratio achieved was 1:10

In [16]:
import numpy as np
from sklearn.decomposition import PCA

# Parameters
variance_threshold = 0.9999  # Retain 100% of variance

# Filter each chunk by the desired column label and value
for chunk in  pd.read_csv("/content/filtered_data.csv", chunksize=300):
  clstr = chunk.drop(columns=['cluster_label']) ;
  btch = clstr.shape[0]
  X =  index_to_embedding(clstr.values.astype(int), pos_encoding)
  X = np.reshape(X , (btch,-1))

  # Initialize PCA with desired variance threshold
  pca = PCA(n_components=variance_threshold, svd_solver='full')
  # Fit PCA to the data
  X_pca = pca.fit_transform(X)

  # Number of principal components required to retain the specified variance threshold
  n_components_required = pca.n_components_

  # Total variance retained
  total_variance_retained = np.sum(pca.explained_variance_ratio_)

  # Print the number of principal components required and total variance retained
  print("Number of Principal Components Required:", n_components_required)
  print("Total Variance Retained:", total_variance_retained)

  # Initialize PCA with desired number of components
  n_components = n_components_required
  pca = PCA(n_components=n_components)

  # Fit PCA to the data and transform the data
  X_pca = pca.fit_transform(X)

  # Print original data shape and transformed data shape
  print("Original Data Shape:", X.shape)
  print("Transformed Data Shape:", X_pca.shape)

  # Print explained variance ratio
  #print("Explained Variance Ratio:", pca.explained_variance_ratio_)

  # Perform inverse transform
  X_inverse = pca.inverse_transform(X_pca)

  # Print data loss
  print("\nDifference between actual embedding and retrived embedding:")
  print(np.sum(np.absolute(X-X_inverse)))

  X_inverse = np.reshape(X_inverse, (btch,-1,1,4) )

  reconstructed_index = embedding_to_index(X_inverse, pos_encoding)
  reconstructed_index = np.reshape(reconstructed_index, (btch,-1))
  print("error on final data : ",np.sum(np.absolute(clstr.values-reconstructed_index)))



Number of Principal Components Required: 70
Total Variance Retained: 1.0
Original Data Shape: (71, 15000)
Transformed Data Shape: (71, 70)

Difference between actual embedding and retrived embedding:
0.5056117
error on final data :  0


Converting label back to data

In [19]:
def dec2bin(dec):
  bin_str = format(int(dec),'b')
  return bin_str

binary_array = np.unpackbits(np.array(reconstructed_index, dtype=np.uint8), axis=1)
print(binary_array)

[[1 0 0 ... 1 0 1]
 [0 1 0 ... 0 0 1]
 [1 1 1 ... 0 0 0]
 ...
 [1 1 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [1 0 1 ... 1 0 0]]


# conclusion

Actual size of data = 10M x 30K matrix

Proposed model
- array final storage = 10M x 300

- Overall compression ratio = 1 : 70

- Can be scaled similarly on to the rows to compress 10M rows

- Size of Look up matrix for embedding 256 x 4

- Size of cluster look up matrix = 1 x 10M

- Size of PCA object for each cluster = 0.35 KB

### Retrieval time

- time complexity = O(log n)
